In [17]:
import pandas as pd
import numpy as np
import os
import time
from openai import OpenAI # Import OpenAI library
import json # To potentially parse structured LLM output
from dotenv import load_dotenv
import openpyxl


In [13]:
# --- IMPORTANT: API Key Configuration ---
# Best practice: Store your key in an environment variable or secure config
# os.environ['OPENAI_API_KEY'] = 'YOUR_ACTUAL_API_KEY' # <-- NEVER hardcode keys directly in scripts shared or committed
# client = OpenAI() # Reads key from environment variable automatically


In [14]:
load_dotenv()


True

In [15]:
try:
    client = OpenAI()
    # Optional: Verify
    if os.getenv('OPENAI_API_KEY'):
        print("OpenAI client initialized after loading .env file.")
    else:
        print("Warning: OPENAI_API_KEY not found after loading .env. Check .env file location and content.")
    # You can now use the 'client' object
    # client.models.list() # Example call
except Exception as e:
    print(f"Error initializing OpenAI client after loading .env: {e}")

OpenAI client initialized after loading .env file.


In [16]:
client = OpenAI()
client.models.list()

SyncPage[Model](data=[Model(id='gpt-4o-audio-preview-2024-12-17', created=1734034239, object='model', owned_by='system'), Model(id='dall-e-3', created=1698785189, object='model', owned_by='system'), Model(id='dall-e-2', created=1698798177, object='model', owned_by='system'), Model(id='gpt-4o-audio-preview-2024-10-01', created=1727389042, object='model', owned_by='system'), Model(id='gpt-4-turbo-preview', created=1706037777, object='model', owned_by='system'), Model(id='text-embedding-3-small', created=1705948997, object='model', owned_by='system'), Model(id='gpt-4-turbo', created=1712361441, object='model', owned_by='system'), Model(id='gpt-4-turbo-2024-04-09', created=1712601677, object='model', owned_by='system'), Model(id='gpt-4.1-nano', created=1744321707, object='model', owned_by='system'), Model(id='gpt-4.1-nano-2025-04-14', created=1744321025, object='model', owned_by='system'), Model(id='gpt-4o-realtime-preview-2024-10-01', created=1727131766, object='model', owned_by='system')

In [18]:
excel_file_path = 'data/excel_fooddesert/full_literature_list_2024-10-21(analyzing).xlsx' 
output_dir_llm = 'llm_outputs' # Directory to save results
os.makedirs(output_dir_llm, exist_ok=True)


# Load Data

In [19]:
print("Loading data...")
if not os.path.exists(excel_file_path):
    print(f"Error: Excel file not found at {excel_file_path}")
    exit()
try:
    df_lit = pd.read_excel(excel_file_path)
    # Remove 'Unnamed' columns
    unnamed_cols_mask = df_lit.columns.str.contains('Unnamed:')
    df_lit = df_lit.loc[:, ~unnamed_cols_mask]
    print(f"Successfully loaded and cleaned Excel file. Shape: {df_lit.shape}")
except Exception as e:
    print(f"An error occurred while loading the Excel file: {e}")
    exit()

Loading data...
Successfully loaded and cleaned Excel file. Shape: (847, 65)


In [20]:
text_cols_for_llm = ['Article Title', 'Abstract', 'Author Keywords', 'Keywords Plus', 'Country', 'Continent']
for col in text_cols_for_llm:
    if col in df_lit.columns:
        df_lit[col] = df_lit[col].fillna('').astype(str)
    else:
        print(f"Warning: Column '{col}' not found.")
        if col in ['Country', 'Continent']: # Add placeholder if essential geo cols missing
             df_lit[col] = 'Unknown'

# Define LLM Interaction Function

In [21]:
def create_measurement_prompt(title, abstract, keywords):
    # Combine keywords for context
    all_keywords = f"{keywords.get('Author Keywords', '')}; {keywords.get('Keywords Plus', '')}"
    prompt = f"""Analyze the following research paper abstract about food deserts. Focus ONLY on how food deserts or food access were measured or assessed.

    Paper Title: {title}
    Keywords: {all_keywords}
    Abstract: {abstract}

    Instructions:
    1. Identify the primary methodology used for measurement.
    2. Categorize this methodology into ONE of the following types:
       - Spatial: Distance/Buffer
       - Spatial: Network Analysis
       - Spatial: Density Metrics (e.g., store counts, RFEI/mRFEI)
       - Temporal Analysis (incorporating time)
       - Socioeconomic Integration (explicitly combining spatial with income, race, vehicle access etc.)
       - Survey/Qualitative Methods
       - Integrated/Mixed-Methods (combining multiple distinct types)
       - Not Applicable/Not Specified
       - Other (please specify briefly)
    3. List the key variables or data types explicitly mentioned as being used in the measurement (e.g., 'supermarket locations', 'census tract income', 'travel time', 'road network', 'household survey data'). List up to 5 key variables.

    Provide the output ONLY in JSON format like this:
    {{
      "methodology_category": "Your Category Here",
      "key_variables": ["Variable 1", "Variable 2", ...]
    }}
    """
    return prompt


In [22]:
def create_intervention_prompt(title, abstract, keywords):
    all_keywords = f"{keywords.get('Author Keywords', '')}; {keywords.get('Keywords Plus', '')}"
    prompt = f"""Analyze the following research paper abstract about food deserts. Focus ONLY on interventions, policies, solutions, or strategies mentioned to address food deserts or improve food access.

    Paper Title: {title}
    Keywords: {all_keywords}
    Abstract: {abstract}

    Instructions:
    1. List the specific interventions, policies, or solutions mentioned (e.g., 'SNAP participation', 'supermarket subsidies', 'mobile markets', 'community gardens', 'nutrition education', 'transport improvements').
    2. If no interventions are mentioned, state "None".

    Provide the output ONLY in JSON format like this:
    {{
      "interventions_mentioned": ["Intervention 1", "Intervention 2", ...] or ["None"]
    }}
    """
    return prompt

In [23]:
def create_regional_prompt(title, abstract, keywords, country, continent):
     all_keywords = f"{keywords.get('Author Keywords', '')}; {keywords.get('Keywords Plus', '')}"
     prompt = f"""Analyze the following research paper abstract about food deserts, specifically considering its regional context ({country}, {continent}).

     Paper Title: {title}
     Keywords: {all_keywords}
     Abstract: {abstract}

     Instructions:
     1. Briefly summarize (1-2 sentences) the main finding or focus of this study AS IT RELATES to its specific location or context, if mentioned.
     2. Does the abstract highlight any unique regional factors (e.g., specific policies, demographic issues like aging, urban form, role of informal markets) influencing food deserts in this location? If yes, list keywords or a brief phrase. If no, state "None".

     Provide the output ONLY in JSON format like this:
     {{
       "context_summary": "Brief summary here.",
       "unique_regional_factors": ["Factor 1 phrase", "Factor 2 phrase", ...] or ["None"]
     }}
     """
     return prompt

In [24]:
def get_llm_analysis(prompt_text, model="gpt-4o-mini", max_retries=3, delay=1):
    """Calls the OpenAI API with specified prompt and handles retries."""
    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[
                    # {"role": "system", "content": "You are a helpful research assistant analyzing abstracts."}, # Optional system message
                    {"role": "user", "content": prompt_text}
                ],
                temperature=0.2, # Lower temperature for more focused, less creative output
                max_tokens=200, # Adjust as needed
                response_format={ "type": "json_object" } # Request JSON output
            )
            # Extract the content and parse JSON
            content = response.choices[0].message.content
            return json.loads(content) # Parse the JSON string response
        except Exception as e:
            print(f"API Error: {e}. Attempt {attempt + 1} of {max_retries}.")
            if attempt < max_retries - 1:
                time.sleep(delay * (attempt + 1)) # Exponential backoff
            else:
                print("Max retries reached. Skipping this paper.")
                return None # Return None or an error indicator

# LLM Analysis (Sample)

In [25]:
sample_df = df_lit.head(10).copy() # Start with first 10 papers
# sample_df = df_lit.iloc[10:20].copy() # Or another slice
# Or process the whole dataframe (use with caution re: cost/time)
# sample_df = df_lit.copy()

results_measurement = []
results_intervention = []
results_regional = []

In [26]:
run_measurement_analysis = True
run_intervention_analysis = True
run_regional_analysis = True

In [27]:
print(f"Processing {len(sample_df)} papers...")
start_time = time.time()

Processing 10 papers...


In [28]:
for index, row in sample_df.iterrows():
    print(f"\nProcessing paper index: {index}...")
    title = row.get('Article Title', '')
    abstract = row.get('Abstract', '')
    keywords = {'Author Keywords': row.get('Author Keywords', ''), 'Keywords Plus': row.get('Keywords Plus', '')}
    country = row.get('Country', 'Unknown')
    continent = row.get('Continent', 'Unknown')

    # Create prompts
    if run_measurement_analysis:
        meas_prompt = create_measurement_prompt(title, abstract, keywords)
        meas_result = get_llm_analysis(meas_prompt)
        results_measurement.append(meas_result)
        print(f"  Measurement Result: {meas_result}")
        time.sleep(1) # Pause between API calls to avoid rate limits

    if run_intervention_analysis:
        int_prompt = create_intervention_prompt(title, abstract, keywords)
        int_result = get_llm_analysis(int_prompt)
        results_intervention.append(int_result)
        print(f"  Intervention Result: {int_result}")
        time.sleep(1)

    if run_regional_analysis:
         reg_prompt = create_regional_prompt(title, abstract, keywords, country, continent)
         reg_result = get_llm_analysis(reg_prompt)
         results_regional.append(reg_result)
         print(f"  Regional Result: {reg_result}")
         time.sleep(1)


Processing paper index: 0...
  Measurement Result: {'methodology_category': 'Spatial: Distance/Buffer', 'key_variables': ['shopping behavior', 'travel distance', 'vendor visits', 'supermarket vendor', 'WIC Program participants']}
  Intervention Result: {'interventions_mentioned': ['None']}
  Regional Result: {'context_summary': 'The study investigates the shopping behaviors of WIC Program participants in food deserts within the Greater Los Angeles area, finding that their shopping patterns are similar to those of participants outside food deserts.', 'unique_regional_factors': ['Greater Los Angeles area', 'WIC Program', 'food deserts']}

Processing paper index: 1...
  Measurement Result: {'methodology_category': 'Spatial: Density Metrics (e.g., store counts, RFEI/mRFEI)', 'key_variables': ['census blocks', 'pricing strategies', 'location', 'ownership', 'restaurant characteristics']}
  Intervention Result: {'interventions_mentioned': ['None']}
  Regional Result: {'context_summary': 'The

In [29]:
if run_measurement_analysis:
    # Extract specific fields from JSON results carefully
    sample_df['LLM_Methodology'] = [res.get('methodology_category', 'Error') if res else 'Error' for res in results_measurement]
    sample_df['LLM_Variables'] = [res.get('key_variables', []) if res else [] for res in results_measurement]

if run_intervention_analysis:
    sample_df['LLM_Interventions'] = [res.get('interventions_mentioned', []) if res else [] for res in results_intervention]

if run_regional_analysis:
     sample_df['LLM_Context_Summary'] = [res.get('context_summary', 'Error') if res else 'Error' for res in results_regional]
     sample_df['LLM_Regional_Factors'] = [res.get('unique_regional_factors', []) if res else [] for res in results_regional]


print("\nRemember to CRITICALLY REVIEW and VERIFY the LLM's output against the original papers before finalizing your tables.")


In [30]:
if run_regional_analysis and 'LLM_Context_Summary' in sample_df.columns:
     print("\nRegional Context Summaries and Factors:")
     # Group by continent and review LLM summaries and factors
     for continent, group in sample_df.groupby('Continent'):
         if continent != 'Unknown':
             print(f"\n--- Region: {continent} ---")
             print(f"  Number of papers in sample: {len(group)}")
             # Display sample summaries/factors for review
             print(group[['Article Title', 'LLM_Context_Summary', 'LLM_Regional_Factors']].head(3))
             # Use this grouped info + LLM output to write the regional comparison table



Regional Context Summaries and Factors:

--- Region: North America ---
  Number of papers in sample: 9
                                       Article Title  \
0  Food Access, Food Deserts, and the Women, Infa...   
1  Fast Food Restaurant Pricing Strategies in Mic...   
2  Vulnerable populations in food deserts: a case...   

                                 LLM_Context_Summary  \
0  The study investigates the shopping behaviors ...   
1  The study investigates how fast food restauran...   
2  The study focuses on the challenges faced by v...   

                                LLM_Regional_Factors  
0  [Greater Los Angeles area, WIC Program, food d...  
1  [Michigan-specific pricing strategies, food de...  
2  [economic disenfranchisement, social disenfran...  

--- Region: South America ---
  Number of papers in sample: 1
                                       Article Title  \
8  Social inequalities in the surrounding areas o...   

                                 LLM_Context_Summa

# LLM Analysis (All)

In [31]:
papers_to_process = df_lit.copy()
total_papers = len(papers_to_process)
print(f"Total papers to process: {total_papers}")

Total papers to process: 847


In [32]:
run_measurement_analysis = True
run_intervention_analysis = True
run_regional_analysis = True
# Saving configuration
save_interval = 50  # Save progress every 50 papers
api_call_delay = 1  # Seconds to wait between API calls (increase if hitting rate limits)
# Output file for progress saving
progress_output_filename = os.path.join(output_dir_llm, 'llm_analysis_full_results_PROGRESS.xlsx')


In [33]:
results_measurement = [None] * total_papers
results_intervention = [None] * total_papers
results_regional = [None] * total_papers

In [35]:
print(f"\nStarting LLM processing for {total_papers} papers...")
print(f"Progress will be saved every {save_interval} papers to {progress_output_filename}")
print(f"Estimated time depends heavily on API response times and delays ({api_call_delay}s between calls).")
start_time_full = time.time()

# --- Main Processing Loop ---
for index, row in papers_to_process.iterrows():
    # Get the integer position for storing results correctly in lists
    list_index = papers_to_process.index.get_loc(index)

    # --- Reporting Progress ---
    if (list_index + 1) % 10 == 0: # Print status update every 10 papers
        elapsed_time = time.time() - start_time_full
        print(f"  Processing paper {list_index + 1}/{total_papers}... (Elapsed: {elapsed_time:.1f}s)")

    # --- Extract data for prompts ---
    title = row.get('Article Title', '')
    abstract = row.get('Abstract', '')
    # Ensure keywords are passed as a dictionary or adjust prompt functions
    keywords = {'Author Keywords': row.get('Author Keywords', ''), 'Keywords Plus': row.get('Keywords Plus', '')}
    country = row.get('Country', 'Unknown')
    continent = row.get('Continent', 'Unknown')

    # --- Call LLM for each analysis type ---
    # Wrap calls in try-except blocks within the loop if needed for more granular error handling

    if run_measurement_analysis:
        if list_index < len(results_measurement): # Check index bounds
             meas_prompt = create_measurement_prompt(title, abstract, keywords)
             meas_result = get_llm_analysis(meas_prompt, delay=api_call_delay)
             results_measurement[list_index] = meas_result # Store result at correct position
             time.sleep(api_call_delay) # Pause between API calls
        else: print(f"Warning: Index {list_index} out of bounds for results_measurement")


    if run_intervention_analysis:
         if list_index < len(results_intervention):
             int_prompt = create_intervention_prompt(title, abstract, keywords)
             int_result = get_llm_analysis(int_prompt, delay=api_call_delay)
             results_intervention[list_index] = int_result # Store result
             time.sleep(api_call_delay)
         else: print(f"Warning: Index {list_index} out of bounds for results_intervention")


    if run_regional_analysis:
         if list_index < len(results_regional):
             reg_prompt = create_regional_prompt(title, abstract, keywords, country, continent)
             reg_result = get_llm_analysis(reg_prompt, delay=api_call_delay)
             results_regional[list_index] = reg_result # Store result
             time.sleep(api_call_delay)
         else: print(f"Warning: Index {list_index} out of bounds for results_regional")


    # --- Periodic Saving ---
    if (list_index + 1) % save_interval == 0 or (list_index + 1) == total_papers: # Save every N or at the very end
        print(f"\n--- Saving progress at paper {list_index + 1} ---")
        # Add results collected SO FAR to a temporary DataFrame for saving
        temp_save_df = papers_to_process.copy() # Start with original data

        # --- Add LLM result columns to the temp dataframe ---
        # Use helper function to safely extract from potentially None results
        def safe_get(result, key, default='Error'):
            return result.get(key, default) if isinstance(result, dict) else default

        def safe_get_list(result, key, default=[]):
            # Return list as string for Excel compatibility
            return str(result.get(key, default)) if isinstance(result, dict) else str(default)

        if run_measurement_analysis:
             temp_save_df['LLM_Methodology'] = [safe_get(res, 'methodology_category') for res in results_measurement]
             temp_save_df['LLM_Variables'] = [safe_get_list(res, 'key_variables') for res in results_measurement]
        if run_intervention_analysis:
             temp_save_df['LLM_Interventions'] = [safe_get_list(res, 'interventions_mentioned') for res in results_intervention]
        if run_regional_analysis:
             temp_save_df['LLM_Context_Summary'] = [safe_get(res, 'context_summary') for res in results_regional]
             temp_save_df['LLM_Regional_Factors'] = [safe_get_list(res, 'unique_regional_factors') for res in results_regional]
        # --- End of adding LLM result columns ---

        try:
            # Select only relevant columns + LLM outputs to save space if needed
            cols_to_save = ['UT (Unique WOS ID)', 'Authors', 'Publication Year', 'Article Title', 'Abstract', 'Author Keywords', 'Keywords Plus', 'Country', 'Continent', 'Research site'] + [col for col in temp_save_df.columns if col.startswith('LLM_')]
            # Ensure all columns exist before trying to save
            cols_to_save = [col for col in cols_to_save if col in temp_save_df.columns]
            temp_save_df[cols_to_save].to_excel(progress_output_filename, index=False, engine='openpyxl')
            print(f"Progress saved to {progress_output_filename}")
        except Exception as e:
            print(f"Error saving progress: {e}")
        print("--- Resuming processing ---")


# --- Final Processing after Loop ---
end_time_full = time.time()
total_duration_minutes = (end_time_full - start_time_full) / 60
print(f"\n>>> Finished processing {total_papers} papers in {total_duration_minutes:.2f} minutes. <<<")


Starting LLM processing for 847 papers...
Progress will be saved every 50 papers to llm_outputs\llm_analysis_full_results_PROGRESS.xlsx
Estimated time depends heavily on API response times and delays (1s between calls).
  Processing paper 10/847... (Elapsed: 56.2s)
  Processing paper 20/847... (Elapsed: 126.8s)
  Processing paper 30/847... (Elapsed: 193.7s)
  Processing paper 40/847... (Elapsed: 259.2s)
  Processing paper 50/847... (Elapsed: 323.4s)

--- Saving progress at paper 50 ---
Progress saved to llm_outputs\llm_analysis_full_results_PROGRESS.xlsx
--- Resuming processing ---
  Processing paper 60/847... (Elapsed: 388.9s)
  Processing paper 70/847... (Elapsed: 454.9s)
  Processing paper 80/847... (Elapsed: 516.5s)
  Processing paper 90/847... (Elapsed: 583.6s)
  Processing paper 100/847... (Elapsed: 645.5s)

--- Saving progress at paper 100 ---
Progress saved to llm_outputs\llm_analysis_full_results_PROGRESS.xlsx
--- Resuming processing ---
  Processing paper 110/847... (Elapsed

# Display the results